# Federated Learning 과 LLM 을 결합한 코드 실습 

Distilgpt2 모델을 활용하여 간단한 LLM federated learning 구현 

학습의 효율성을 위해 7B 의 모델을 BASE 모델로 선정. 

데이터셋의 구성은 2가지 다른 도메인 데이터셋 선정. 

1. 의료 (MIMIC-III), 2. 법률

다만 이 파일에서는 법률 도메인 데이터셋으로만 성능을 평가 할 것.

알고리즘은 다음과 같은 알고리즘을 사용

1. FedAvg 2. FedProx 3. FedNova 4. FedOpt 5. SCAFFOLD

Evalution 지표는 다음과 같음
1. 데이터셋에 따른 평가지표
2. Communication Cost

클라이언트 모델 개수 : 5



# 1. 베이스 모델 선정


In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "distilgpt2" 

# 평가 비교를 위한 베이스라인 모델 
baseline_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)



# 2. LoRA적용


In [3]:
from peft import get_peft_model, LoraConfig, TaskType

# Apply LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,  # Lower rank for LoRA with smaller model
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["attn.c_attn", "attn.c_proj"], 
)
model = get_peft_model(model, lora_config)
model.to(device)

baseline_model = get_peft_model(baseline_model, lora_config).to(device)


# Debug print to verify LoRA layers
print("Verifying LoRA applied layers:")
for name, param in model.named_parameters():
    if "lora" in name or any(layer in name for layer in lora_config.target_modules):
        print(f"Layer: {name}, requires_grad: {param.requires_grad}, shape: {param.shape}")

Verifying LoRA applied layers:
Layer: base_model.model.transformer.h.0.attn.c_attn.base_layer.weight, requires_grad: False, shape: torch.Size([768, 2304])
Layer: base_model.model.transformer.h.0.attn.c_attn.base_layer.bias, requires_grad: False, shape: torch.Size([2304])
Layer: base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight, requires_grad: True, shape: torch.Size([8, 768])
Layer: base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight, requires_grad: True, shape: torch.Size([2304, 8])
Layer: base_model.model.transformer.h.0.attn.c_proj.base_layer.weight, requires_grad: False, shape: torch.Size([768, 768])
Layer: base_model.model.transformer.h.0.attn.c_proj.base_layer.bias, requires_grad: False, shape: torch.Size([768])
Layer: base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight, requires_grad: True, shape: torch.Size([8, 768])
Layer: base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight, requires_grad: True, shape: torch.Size([7

/home/park/.local/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


# 3. 특정 도메인 데이터셋 준비

In [4]:
from datasets import load_dataset

# Load legal dataset
legal_dataset = load_dataset("lex_glue", "ecthr_a")  # European Court of Human Rights dataset



In [5]:
print(legal_dataset.column_names)


{'train': ['text', 'labels'], 'test': ['text', 'labels'], 'validation': ['text', 'labels']}


In [6]:
# Set padding token for tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

global_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def tokenize_legal_data(batch):
    texts = [str(text) for text in batch['text']]
    tokenized_output = tokenizer(texts, padding="max_length", truncation=True, max_length=128)
    return tokenized_output

# Tokenize the legal dataset
legal_dataset = legal_dataset.map(tokenize_legal_data, batched=True)


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# 4. 데이터셋은 subset 으로 나누기

In [7]:
import numpy as np
from torch.utils.data import Dataset

class FederatedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.dataset[idx]["input_ids"]),
            "attention_mask": torch.tensor(self.dataset[idx]["attention_mask"]),
        }
    
def split_dataset(dataset, num_clients=5):
    client_datasets = []
    split_indices = np.array_split(np.arange(len(dataset)), num_clients)
    for indices in split_indices:
        client_datasets.append(dataset.select(indices))
    return client_datasets

# Split the legal dataset for federated learning
legal_client_datasets = split_dataset(legal_dataset['train'], num_clients=5)


# 5. 각 클라이언트 모델의 data loader설정

In [8]:
from torch.utils.data import DataLoader

# Create DataLoader for each client dataset
def create_data_loader(client_dataset, batch_size=8):
    torch_dataset = FederatedDataset(client_dataset)
    return DataLoader(torch_dataset, batch_size=batch_size, shuffle=True)

# Initialize DataLoader for the legal dataset
legal_client_loaders = [create_data_loader(client_dataset) for client_dataset in legal_client_datasets]


# 6. Federated Learning 학습을 위한 환경 설정 




In [9]:
from torch.optim import AdamW
from tqdm import tqdm

# Define Client Class
class Client:
    def __init__(self, client_id, model, train_loader, device):
        self.client_id = client_id
        self.model = model
        self.train_loader = train_loader
        self.device = device

    def train(self, epochs=3, lr=5e-5):
        self.model.train()
        optimizer = AdamW(self.model.parameters(), lr=lr)
        total_loss = 0
        
        for epoch in range(epochs):
            epoch_loss = 0
            for batch in tqdm(self.train_loader, desc=f"Client {self.client_id} Training", leave=False):
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=input_ids)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
            total_loss += epoch_loss / len(self.train_loader)
            print(f"Client {self.client_id} - Epoch {epoch + 1} Loss: {epoch_loss / len(self.train_loader)}")
        
        return {k: v.cpu() for k, v in self.model.state_dict().items()}


# 통합 알고리즘 정의

# FedAVG 함수

In [10]:
import copy
import torch

class FedAvg:
    def aggregate(self, global_model, local_models):
        new_model = copy.deepcopy(global_model)
        
        # Average parameters across all local models
        for name, param in new_model.named_parameters():
            param.data = torch.mean(torch.stack([local_model[name].data for local_model in local_models]), dim=0)
        
        return new_model.state_dict()


# FedProx 함수 

In [11]:
class FedProx:
    def __init__(self, mu=0.01):
        self.mu = mu

    def aggregate(self, global_model, local_models):
        new_model = copy.deepcopy(global_model)
        
        # Aggregate parameters with Proximal regularization
        for name, param in new_model.named_parameters():
            avg_param = torch.mean(torch.stack([local_model[name].data for local_model in local_models]), dim=0)
            prox_term = self.mu * (avg_param - global_model.state_dict()[name].data)
            param.data = avg_param - prox_term
        
        return new_model.state_dict()


# SCAFFOLD 함수

In [12]:
class SCAFFOLD:
    def __init__(self):
        self.server_control = {}

    def aggregate(self, global_model, local_models, client_controls):
        new_model = copy.deepcopy(global_model)
        
        # Aggregate parameters considering control variates
        for name, param in new_model.named_parameters():
            control_updates = torch.stack([
                client_controls[i][name].data + local_models[i][name].data - global_model.state_dict()[name].data
                for i in range(len(local_models))
            ])
            param.data += torch.mean(control_updates, dim=0)
        
        return new_model.state_dict()


# FedAdam 함수

In [13]:
class FedYogi:
    def __init__(self, eta=1e-3, tau=1e-3):
        self.eta = eta
        self.tau = tau
        self.v = {}

    def aggregate(self, global_model, local_models):
        new_model = copy.deepcopy(global_model)
        
        for name, param in new_model.named_parameters():
            g = torch.mean(torch.stack([
                local_model[name].data - global_model.state_dict()[name].data for local_model in local_models
            ]), dim=0)
            
            # Initialize v if it doesn’t exist for this parameter
            if name not in self.v:
                self.v[name] = torch.zeros_like(param.data)
                
            self.v[name] += torch.sign(self.v[name] - g) * g**2
            param.data -= self.eta * g / (torch.sqrt(self.v[name]) + self.tau)
        
        return new_model.state_dict()


In [14]:
aggregators = {
    "FedAvg": FedAvg(),
    "FedProx": FedProx(mu=0.01),
    "SCAFFOLD": SCAFFOLD(),
    "FedYogi": FedYogi(eta=1e-3, tau=1e-3)
}

In [15]:
from torch.utils.data import DataLoader

def prepare_data_loader(dataset, split, batch_size=32):
    # Ensure the specified split exists in the dataset
    if split not in dataset:
        raise ValueError(f"Split '{split}' not found in dataset.")
    
    # Load the split and remove any None entries
    filtered_dataset = dataset[split].filter(lambda x: x['input_ids'] is not None)
    
    # Return a DataLoader for the filtered dataset
    return DataLoader(filtered_dataset, batch_size=batch_size, shuffle=True)

# Example usage
legal_train_loader = prepare_data_loader(legal_dataset, 'train')


Filter:   0%|          | 0/9000 [00:00<?, ? examples/s]

In [16]:
# Custom collate function for padding and label creation
def custom_collate_fn(batch):
    # Convert each dictionary item to a batch
    inputs = [item["input_ids"] for item in batch]
    masks = [item["attention_mask"] for item in batch]

    # Pad sequences to the max length in the batch
    inputs = tokenizer.pad({"input_ids": inputs, "attention_mask": masks}, return_tensors="pt")

    # Create labels by duplicating input_ids and masking padding tokens for labels
    labels = inputs["input_ids"].clone()
    labels[labels == tokenizer.pad_token_id] = -100  # Mask padding tokens in labels

    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": labels}


# 연합학습 메서드 정의

In [17]:
# Define federated training function for the legal dataset
def federated_training(global_model, client_datasets, aggregator_name="FedAvg", rounds=5, local_epochs=1):
    if aggregator_name == "FedAvg":
        aggregator = FedAvg()
    else:
        raise ValueError("Unknown aggregator")

    for round_num in range(rounds):
        print(f"--- Round {round_num + 1}/{rounds} ---")
        client_weights = []
        
        for client_idx, client_data in enumerate(client_datasets):
            print(f"Processing client {client_idx + 1}/{len(client_datasets)}")
            local_model = global_model.to(device)
            local_model.train()

            data_loader = DataLoader(FederatedDataset(client_data), batch_size=8, shuffle=True)
            optimizer = torch.optim.SGD(local_model.parameters(), lr=0.001)

            for epoch in range(local_epochs):
                for batch in tqdm(data_loader, desc=f"Client {client_idx + 1} Training", leave=False):
                    input_ids = batch["input_ids"].to(device)
                    attention_mask = batch["attention_mask"].to(device)
                    labels = batch["input_ids"].to(device)

                    optimizer.zero_grad()
                    outputs = local_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    loss.backward()
                    optimizer.step()

            client_weights.append(local_model.state_dict())

        if client_weights:
            global_model.load_state_dict(aggregator.aggregate(global_model, client_weights))

    return global_model

# Initialize global model for federated learning
global_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
global_model = get_peft_model(global_model, lora_config).to(device)

# Split the legal dataset into client datasets for federated learning
legal_client_datasets = split_dataset(legal_dataset['train'], num_clients=5)

# Train the federated model on the legal dataset
print("Starting Federated Training on Legal Dataset...")
trained_legal_model = federated_training(global_model, legal_client_datasets)
print("Federated Training on Legal Dataset Complete.")


Starting Federated Training on Legal Dataset...
--- Round 1/5 ---
Processing client 1/5


Processing client 2/5


Processing client 3/5


Processing client 4/5


Processing client 5/5


...

Processing client 2/5


Processing client 3/5


Processing client 4/5


Processing client 5/5


Federated Training on Legal Dataset Complete.


# 평가 비교를 위한 기존 모델이 ft 한 모델 생성

In [18]:
# Fine-tune LoRA model directly on the full legal dataset (baseline)
def fine_tune_lora_model(model, dataset, tokenizer, device, epochs=3, batch_size=8, lr=5e-5):
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr)
    dataloader = DataLoader(FederatedDataset(dataset['train']), batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Fine-tuning Epoch {epoch + 1}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1} Loss: {avg_loss:.4f}")

    return model

# Initialize and fine-tune baseline model on the legal dataset
print("Starting LoRA Fine-Tuning on Legal Dataset...")
baseline_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
baseline_model = get_peft_model(baseline_model, lora_config).to(device)
fine_tuned_model = fine_tune_lora_model(baseline_model, legal_dataset, tokenizer, device)
print("LoRA Fine-Tuning Complete.")


Starting LoRA Fine-Tuning on Legal Dataset...


Fine-tuning Epoch 1: 100%|██████████| 1125/1125 [00:31<00:00, 35.62it/s]


Epoch 1 Loss: 3.0996


Fine-tuning Epoch 2: 100%|██████████| 1125/1125 [00:31<00:00, 35.62it/s]


Epoch 2 Loss: 2.8646


Fine-tuning Epoch 3: 100%|██████████| 1125/1125 [00:31<00:00, 35.61it/s]

Epoch 3 Loss: 2.8091
LoRA Fine-Tuning Complete.


# baseline 모델과 local 모델 성능 비교

In [21]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import torch.nn.functional as F

# Define evaluation function with multiple metrics
def evaluate(model, eval_loader, tokenizer, device):
    model.eval()
    total_loss = 0
    num_batches = 0
    total_accuracy = 0
    total_bleu = 0
    total_rouge = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    
    rouge_scorer_obj = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    with torch.no_grad():
        for batch in eval_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            
            # Calculate Loss
            loss = outputs.loss
            total_loss += loss.item()
            
            # Calculate Perplexity
            perplexity = torch.exp(torch.tensor(total_loss / (num_batches + 1)))
            
            # Calculate Accuracy
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct = (predictions == input_ids).float() * attention_mask  # only count valid tokens
            accuracy = correct.sum() / attention_mask.sum()
            total_accuracy += accuracy.item()

            # Calculate BLEU Score
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
            batch_bleu = [sentence_bleu([ref.split()], pred.split()) for ref, pred in zip(decoded_labels, decoded_preds)]
            total_bleu += sum(batch_bleu) / len(batch_bleu)

            # Calculate ROUGE Score
            batch_rouge = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
            for ref, pred in zip(decoded_labels, decoded_preds):
                scores = rouge_scorer_obj.score(ref, pred)
                for key in batch_rouge:
                    batch_rouge[key] += scores[key].fmeasure
            for key in total_rouge:
                total_rouge[key] += batch_rouge[key] / len(decoded_labels)
            
            num_batches += 1

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches
    avg_bleu = total_bleu / num_batches
    avg_rouge = {key: total_rouge[key] / num_batches for key in total_rouge}
    
    return {
        "Loss": avg_loss,
        "Perplexity": perplexity.item(),
        "Accuracy": avg_accuracy,
        "BLEU": avg_bleu,
        "ROUGE": avg_rouge
    }


# 평가 지표 결과 비교

In [ ]:
# Create evaluation DataLoader
legal_loader = create_data_loader(legal_dataset['train'], batch_size=8)

# Evaluate the federated model
print("\nEvaluating Federated Model on Legal Dataset...")
fed_metrics = evaluate(trained_legal_model, legal_loader, tokenizer, device)

# Evaluate the baseline fine-tuned LoRA model
print("\nEvaluating Baseline LoRA Fine-Tuned Model on Legal Dataset...")
baseline_metrics = evaluate(fine_tuned_model, legal_loader, tokenizer, device)



Evaluating Federated Model on Legal Dataset...


/home/park/anaconda3/envs/fedllm/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/park/anaconda3/envs/fedllm/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/park/anaconda3/envs/fedllm/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider usi


Evaluating Baseline LoRA Fine-Tuned Model on Legal Dataset...


In [23]:
# Store evaluation results
evaluation_results = {
    "Federated Model": fed_metrics,
    "Baseline LoRA Fine-Tuned Model": baseline_metrics,
}

# Print the comparison results
print("\n--- Comparison Results ---")
for model_name, metrics in evaluation_results.items():
    print(f"\n{model_name} Metrics:")
    print(f"  Loss: {metrics['Loss']:.4f}")
    print(f"  Perplexity: {metrics['Perplexity']:.4f}")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  BLEU Score: {metrics['BLEU']:.4f}")
    print(f"  ROUGE Scores: {metrics['ROUGE']}")



--- Comparison Results ---

Federated Model Metrics:
  Loss: 3.2773
  Perplexity: 26.5046
  Accuracy: 0.0077
  BLEU Score: 0.0503
  ROUGE Scores: {'rouge1': 0.5141208525212896, 'rouge2': 0.17655084431588333, 'rougeL': 0.38030992332709235}

Baseline LoRA Fine-Tuned Model Metrics:
  Loss: 2.6610
  Perplexity: 14.3110
  Accuracy: 0.0004
  BLEU Score: 0.1144
  ROUGE Scores: {'rouge1': 0.5455923194247646, 'rouge2': 0.22871974976791828, 'rougeL': 0.43961231288453784}


# 결과 분석 (FedAVG 알고리즘)

### 1. loss 는 baseline model 미세조정이 더 좋음
### 2. perplexity : baseline LoRA model has a significantly lower perplexity than the federated model
### 3. Accuracy : 두 모델 다 매우 심각하게 낮음
### 4. BLEU 점수 : baseline model has a noticeably higher
### 5. ROUGE : Baseline 모델이 전반적으로 높음 

# 모델 초기화 (다음 알고리즘 실험을 위한 과정)

In [30]:
def initialize_model():
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    return get_peft_model(model, lora_config).to(device)

# 평가지표 저장
evaluation_results = {}


# Federated Learning 함수 (Fedavg 부터 FedYogi 전부 학습 가능한 메서드)

In [36]:
def federated_training(global_model, client_datasets, aggregator, rounds=50, local_epochs=5):
    for round_num in range(rounds):
        print(f"--- Round {round_num + 1}/{rounds} ---")
        client_weights = []
        for client_idx, client_data in enumerate(client_datasets):
            print(f"\nTraining Client {client_idx + 1}/{len(client_datasets)}")
            local_model = copy.deepcopy(global_model).to(device)
            local_model.train()

            data_loader = DataLoader(FederatedDataset(client_data), batch_size=8, shuffle=True)
            optimizer = torch.optim.AdamW(local_model.parameters(), lr=0.001)

            for epoch in range(local_epochs):
                epoch_loss = 0
                for batch in tqdm(data_loader, desc=f"Client {client_idx + 1} Training Epoch {epoch + 1}", leave=False):
                    input_ids = batch["input_ids"].to(device)
                    attention_mask = batch["attention_mask"].to(device)
                    labels = batch["input_ids"].to(device)

                    optimizer.zero_grad()
                    outputs = local_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    loss.backward()
                    optimizer.step()

                    epoch_loss += loss.item()

                avg_epoch_loss = epoch_loss / len(data_loader)
                print(f"Client {client_idx + 1} - Epoch {epoch + 1} Loss: {avg_epoch_loss:.4f}")

            client_weights.append(local_model.state_dict())

        if client_weights:
            if isinstance(aggregator, SCAFFOLD):
                client_controls = [client_state["control"] for client_state in client_weights]
                global_model.load_state_dict(aggregator.aggregate(global_model, client_weights, client_controls))
            else:
                global_model.load_state_dict(aggregator.aggregate(global_model, client_weights))

    return global_model

In [ ]:
# Split the dataset into client datasets
legal_client_datasets = split_dataset(legal_dataset['train'], num_clients=5)
legal_loader = create_data_loader(legal_dataset['train'], batch_size=8)

# Define aggregation algorithms
aggregators = {
    "FedAvg": FedAvg(),
    "FedProx": FedProx(mu=0.01),
    "SCAFFOLD": SCAFFOLD(),
    "FedYogi": FedYogi(eta=1e-3, tau=1e-3),
}

# Dictionary to store evaluation results
evaluation_results = {}

# Train and evaluate for each aggregation algorithm
for name, aggregator in aggregators.items():
    print(f"\n--- Training with {name} Aggregation ---")
    federated_model = initialize_model()
    trained_model = federated_training(federated_model, legal_client_datasets, aggregator)

    # Evaluate the trained model
    print(f"\nEvaluating Model Trained with {name} on Legal Dataset...")
    metrics = evaluate(trained_model, legal_loader, tokenizer, device)
    evaluation_results[name] = metrics
    print(f"{name} Aggregation - Metrics: {metrics}")



--- Training with FedAvg Aggregation ---
--- Round 1/50 ---

Training Client 1/5


Client 1 - Epoch 1 Loss: 2.9468


Client 1 - Epoch 2 Loss: 2.6423


Client 1 - Epoch 3 Loss: 2.5335


Client 1 - Epoch 4 Loss: 2.4664


...

Client 4 - Epoch 2 Loss: 2.3772


Client 4 - Epoch 3 Loss: 2.3633


Client 4 - Epoch 4 Loss: 2.3520


Client 4 - Epoch 5 Loss: 2.3397

Training Client 5/5


Client 5 Training Epoch 1:  37%|███▋      | 84/225 [00:02<00:03, 35.58it/s]

In [ ]:
print("\n--- Aggregation Comparison Results ---")
for aggregator_name, metrics in evaluation_results.items():
    print(f"\n{aggregator_name} Metrics:")
    print(f"  Loss: {metrics['Loss']:.4f}")
    print(f"  Perplexity: {metrics['Perplexity']:.4f}")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  BLEU Score: {metrics['BLEU']:.4f}")
    print(f"  ROUGE Scores: {metrics['ROUGE']}")
